In [1]:
from dask import dataframe as pd
import numpy as np 
import torch
import matplotlib as plt

In [2]:
patients_df = pd.read_csv("hosp/patients.csv")

In [3]:
patients_df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [2]:
admissions_df = pd.read_csv("hosp/admissions.csv", dtype={'deathtime': 'object'})

In [5]:
admissions_df["hadm_id"]== 29079034

Dask Series Structure:
npartitions=1
    bool
     ...
Name: hadm_id, dtype: bool
Dask Name: eq, 3 graph layers

In [12]:
b = (admissions_df["hadm_id"] == 29079034)
admissions_df[b].compute()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0


In [5]:
admissions_df.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [6]:
provider_df = pd.read_csv("hosp/provider.csv")

In [7]:
provider_df.head()

,provider_id
0,P0003D
1,P000DI
2,P000EW
3,P000H9
4,P000I6


In [8]:
mb_events_df = pd.read_csv("hosp/microbiologyevents.csv", dtype={'ab_name': 'object',
       'dilution_comparison': 'object',
       'dilution_text': 'object',
       'interpretation': 'object',
       'org_name': 'object'})

In [9]:
mb_events_df.head()

,microevent_id,subject_id,hadm_id,micro_specimen_id,order_provider_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,...,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments
0,1,10000032,NaN,636109,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70093,Blood (Toxo),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEGATIVE FOR TOXOPLASMA IgG ANTIBODY BY EIA. ...
1,2,10000032,NaN,1836584,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POSITIVE BY EIA. A positive IgG result genera...
2,3,10000032,NaN,4131591,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70087,Blood (CMV AB),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___
3,4,10000032,NaN,4131591,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70087,Blood (CMV AB),2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEGATIVE FOR CMV IgM ANTIBODY BY EIA. INTERPR...
4,5,10000032,NaN,6028147,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70088,Blood (EBV),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POSITIVE BY EIA.


In [10]:
mb_events_df.columns

Index(['microevent_id', 'subject_id', 'hadm_id', 'micro_specimen_id',
       'order_provider_id', 'chartdate', 'charttime', 'spec_itemid',
       'spec_type_desc', 'test_seq', 'storedate', 'storetime', 'test_itemid',
       'test_name', 'org_itemid', 'org_name', 'isolate_num', 'quantity',
       'ab_itemid', 'ab_name', 'dilution_text', 'dilution_comparison',
       'dilution_value', 'interpretation', 'comments'],
      dtype='object')

In [11]:
lab_events_df = pd.read_csv("hosp/labevents.csv")

In [12]:
lab_events_df.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN


In [13]:
prescriptions_df = pd.read_csv("hosp/prescriptions.csv",dtype={'dose_val_rx': 'object',
       'form_rx': 'object',
       'form_val_disp': 'object',
       'gsn': 'object',
       'ndc': 'float64',
       'poe_seq': 'float64'})

In [14]:
prescriptions_df.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,...,004490,9.041989e+08,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,...,NaN,0.000000e+00,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,P260SK,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,...,008209,5.107901e+10,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,...,063231,6.022761e+06,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,...,006549,6.332303e+10,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


In [15]:
prescriptions_df.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route'],
      dtype='object')

In [16]:
diagnoses_df = pd.read_csv("hosp/diagnoses_icd.csv")

In [17]:
diagnoses_df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [18]:
d_diagnoses_df = pd.read_csv("hosp/d_icd_diagnoses.csv", dtype={'icd_code': 'object'})

In [19]:
d_diagnoses_df.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [20]:
d_diagnoses_df[d_diagnoses_df.loc[:, "icd_code"] == "5723"].compute()

,icd_code,icd_version,long_title
5889,5723,9,Portal hypertension


In [21]:
diagnoses_df["icd_code"].value_counts(normalize=False)

Dask Series Structure:
npartitions=1
    int64
      ...
Name: icd_code, dtype: int64
Dask Name: value-counts-agg, 4 graph layers

In [22]:
d_diagnoses_df[d_diagnoses_df.loc[:, "icd_code"] == "4019"].compute()

,icd_code,icd_version,long_title
4651,4019,9,Unspecified essential hypertension


In [23]:
d_diagnoses_df[d_diagnoses_df["long_title"].str.contains("sepsis")].compute()

,icd_code,icd_version,long_title
7263,67020,9,"Puerperal sepsis, unspecified as to episode of..."
7264,67022,9,"Puerperal sepsis, delivered, with mention of p..."
7265,67024,9,"Puerperal sepsis, postpartum condition or comp..."
9163,77181,9,Septicemia [sepsis] of newborn
12079,99592,9,Severe sepsis
12257,A021,10,Salmonella sepsis
12413,A227,10,Anthrax sepsis
12435,A267,10,Erysipelothrix sepsis
12470,A327,10,Listerial sepsis
12529,A40,10,Streptococcal sepsis


In [24]:
d_hcps = pd.read_csv("hosp/d_hcpcs.csv",dtype={'long_description': 'object'})

In [25]:
d_hcps.head()

,code,category,long_description,short_description
0,00000,NaN,NaN,Invalid Code
1,0001F,2.0,NaN,Composite measures
2,0002F,2.0,NaN,Composite measures
3,0003F,2.0,NaN,Composite measures
4,0004F,2.0,NaN,Composite measures


In [26]:
d_icd_proceures = pd.read_csv("hosp/d_icd_procedures.csv", dtype={'icd_code': 'object'})

In [27]:
d_icd_proceures.head()

,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"


In [28]:
d_lab_items = pd.read_csv("hosp/d_labitems.csv")

In [29]:
d_lab_items.head()

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas


In [30]:
drg_codes = pd.read_csv("hosp/drgcodes.csv")

In [31]:
drg_codes.head()

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,10000032,22595853,APR,283,OTHER DISORDERS OF THE LIVER,2.0,2.0
1,10000032,22595853,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
2,10000032,22841357,APR,279,HEPATIC COMA & OTHER MAJOR ACUTE LIVER DISORDERS,3.0,2.0
3,10000032,22841357,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
4,10000032,25742920,APR,283,OTHER DISORDERS OF THE LIVER,3.0,2.0


In [32]:
emar_df = pd.read_csv("hosp/emar.csv",dtype={'enter_provider_id': 'object',
       'hadm_id': 'float64'})

In [33]:
emar_df.head()

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,enter_provider_id,charttime,medication,event_txt,scheduletime,storetime
0,10000032,22595853.0,10000032-10,10,10000032-36,48770010.0,NaN,2180-05-07 00:44:00,Potassium Chloride,Administered,2180-05-07 00:44:00,2180-05-07 00:44:00
1,10000032,22595853.0,10000032-11,11,10000032-22,14779570.0,NaN,2180-05-07 00:44:00,Sodium Chloride 0.9% Flush,Flushed,2180-05-07 00:44:00,2180-05-07 00:44:00
2,10000032,22595853.0,10000032-12,12,10000032-37,93463122.0,NaN,2180-05-07 06:10:00,Ipratropium Bromide Neb,Administered,2180-05-07 06:00:00,2180-05-07 06:10:00
3,10000032,22595853.0,10000032-13,13,10000032-28,42497745.0,NaN,2180-05-07 05:00:00,Albuterol Inhaler,Administered,2180-05-07 06:29:00,2180-05-07 06:29:00
4,10000032,22595853.0,10000032-14,14,10000032-29,69131933.0,NaN,2180-05-07 07:51:00,Emtricitabine-Tenofovir (Truvada),Administered,2180-05-07 08:00:00,2180-05-07 07:56:00


In [34]:
emar_d_df = pd.read_csv("hosp/emar_detail.csv",dtype={'completion_interval': 'object',
       'continued_infusion_in_other_location': 'object',
       'dose_due': 'object',
       'dose_given': 'object',
       'infusion_complete': 'object',
       'infusion_rate_adjustment': 'object',
       'infusion_rate_unit': 'object',
       'new_iv_bag_hung': 'object',
       'non_formulary_visual_verification': 'object',
       'product_amount_given': 'object',
       'product_description_other': 'object',
       'reason_for_no_barcode': 'object',
       'restart_interval': 'object',
       'route': 'object',
       'side': 'object',
       'site': 'object'})

In [35]:
emar_d_df.head()

,subject_id,emar_id,emar_seq,parent_field_ordinal,administration_type,pharmacy_id,barcode_type,reason_for_no_barcode,complete_dose_not_given,dose_due,...,infusion_rate_unit,route,infusion_complete,completion_interval,new_iv_bag_hung,continued_infusion_in_other_location,restart_interval,side,site,non_formulary_visual_verification
0,10000032,10000032-10,10,1.1,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,10000032-10,10,1.2,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,10000032-10,10,1.3,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,10000032-10,10,1.4,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,10000032-10,10,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
hcpcs_events_df = pd.read_csv("hosp/hcpcsevents.csv")

In [37]:
hcpcs_events_df.head()

,subject_id,hadm_id,chartdate,hcpcs_cd,seq_num,short_description
0,10000068,25022803,2160-03-04,99218,1,Hospital observation services
1,10000084,29888819,2160-12-28,G0378,1,Hospital observation per hr
2,10000108,27250926,2163-09-27,99219,1,Hospital observation services
3,10000117,22927623,2181-11-15,43239,1,Digestive system
4,10000117,22927623,2181-11-15,G0378,2,Hospital observation per hr


In [38]:
omr_df = pd.read_csv("hosp/omr.csv")

In [39]:
omr_df.head()

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15


In [40]:
pharmacy_df = pd.read_csv("hosp/pharmacy.csv",dtype={'expiration_value': 'float64',
       'expirationdate': 'object',
       'fill_quantity': 'object',
       'infusion_type': 'object',
       'sliding_scale': 'object'})

In [41]:
pharmacy_df.head()

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,10000032,22595853,11700683,10000032-34,2180-05-07 01:00:00,2180-05-07 22:00:00,Acetaminophen,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
1,10000032,22595853,14779570,10000032-22,2180-05-07 00:00:00,2180-05-07 22:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,...,NaN,NaN,3.0,NaN,Ongoing,36.0,Hours,NaN,Floor Stock Item,NaN
2,10000032,22595853,19796602,10000032-50,2180-05-08 08:00:00,2180-05-07 22:00:00,Furosemide,Unit Dose,Discontinued via patient discharge,2180-05-07 09:32:35,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
3,10000032,22595853,20256254,10000032-32,2180-05-07 01:00:00,2180-05-07 22:00:00,Raltegravir,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,...,NaN,NaN,2.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
4,10000032,22595853,28781051,10000032-27,2180-05-07 00:00:00,2180-05-07 22:00:00,Heparin,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,...,NaN,NaN,3.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN


In [42]:
pharmacy_df.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'starttime',
       'stoptime', 'medication', 'proc_type', 'status', 'entertime',
       'verifiedtime', 'route', 'frequency', 'disp_sched', 'infusion_type',
       'sliding_scale', 'lockout_interval', 'basal_rate', 'one_hr_max',
       'doses_per_24_hrs', 'duration', 'duration_interval', 'expiration_value',
       'expiration_unit', 'expirationdate', 'dispensation', 'fill_quantity'],
      dtype='object')

In [43]:
services_df = pd.read_csv("hosp/services.csv",dtype={'prev_service': 'object'})

In [44]:
services_df.head()

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10000032,22595853,2180-05-06 22:24:57,NaN,MED
1,10000032,22841357,2180-06-26 18:28:08,NaN,MED
2,10000032,25742920,2180-08-05 23:44:50,NaN,MED
3,10000032,29079034,2180-07-23 12:36:04,NaN,MED
4,10000068,25022803,2160-03-03 23:17:17,NaN,MED


In [45]:
transfer_df = pd.read_csv("hosp/transfers.csv",dtype={'hadm_id': 'float64'})

In [46]:
transfer_df.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,NaN,2180-05-07 17:21:27,NaN
3,10000032,22841357.0,34100253,discharge,NaN,2180-06-27 18:49:12,NaN
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12


In [47]:
procedures_df = pd.read_csv("hosp/procedures_icd.csv")

In [48]:
procedures_df.head()

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


In [49]:
poe_df = pd.read_csv("hosp/poe.csv")

In [50]:
poe_df.head()

,poe_id,poe_seq,subject_id,hadm_id,ordertime,order_type,order_subtype,transaction_type,discontinue_of_poe_id,discontinued_by_poe_id,order_provider_id,order_status
0,10000032-10,10,10000032,22595853,2180-05-06 20:50:49,Lab,NaN,New,NaN,NaN,P9705S,Inactive
1,10000032-100,100,10000032,22841357,2180-06-26 22:25:43,Respiratory,Oxygen Therapy,New,NaN,NaN,P8971Q,Inactive
2,10000032-102,102,10000032,22841357,2180-06-26 22:37:06,Medications,NaN,D/C,10000032-93,NaN,P8971Q,Inactive
3,10000032-103,103,10000032,22841357,2180-06-26 22:37:06,Medications,NaN,Change,10000032-89,NaN,P8971Q,Inactive
4,10000032-104,104,10000032,22841357,2180-06-27 07:06:36,ADT orders,Admit,Change,10000032-75,NaN,P61HH1,Inactive


In [51]:
d_poe_df = pd.read_csv("hosp/poe_detail.csv")

In [52]:
d_poe_df.head()

,poe_id,poe_seq,subject_id,field_name,field_value
0,10000032-104,104,10000032,Admit category,Admit to inpatient
1,10000032-104,104,10000032,Admit to,Medicine
2,10000032-109,109,10000032,Admit category,Admit to inpatient
3,10000032-109,109,10000032,Discharge Planning,Finalized
4,10000032-111,111,10000032,Admit category,Admit to inpatient


In [53]:
icd_counts = diagnoses_df["icd_code"].value_counts(normalize=False).compute()
high_abundant_icd_ds = icd_counts[icd_counts>10000]

In [54]:
high_abundant_icd_ds

4019     97361
2724     63875
I10      51706
E785     48581
53081    46238
         ...  
I129     10178
V1254    10091
V4986    10028
V270     10016
E1122    10003
Name: icd_code, Length: 66, dtype: int64

In [55]:
diagnoses_df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [56]:
uniques, counts = np.unique(diagnoses_df["icd_code"].compute().values, return_counts=True)
high_abundant_icds = uniques[counts > 20_000]

In [57]:
high_abundant_icds.shape

(24,)

In [58]:
d_diagnoses_df.columns

Index(['icd_code', 'icd_version', 'long_title'], dtype='object')

In [59]:
titles = d_diagnoses_df[d_diagnoses_df["icd_code"].isin(high_abundant_icds)]["long_title"].compute()

In [107]:
for i in titles:
    print(i)

Unspecified acquired hypothyroidism
Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled
Pure hypercholesterolemia
Other and unspecified hyperlipidemia
Gout, unspecified
Hyposmolality and/or hyponatremia
Dehydration
Obesity, unspecified
Anemia, unspecified
Anxiety state, unspecified
Alcohol abuse, unspecified
Tobacco use disorder
Depressive disorder, not elsewhere classified
Obstructive sleep apnea (adult)(pediatric)
Other chronic pain
Unspecified essential hypertension
Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage I through stage IV, or unspecified
Old myocardial infarction
Coronary atherosclerosis of unspecified type of vessel, native or graft
Coronary atherosclerosis of native coronary artery
Atrial fibrillation
Congestive heart failure, unspecified
Pneumonia, organism unspecified
Asthma, unspecified type, unspecified
Chronic airway obstruction, not elsewhere classified
Esophageal reflux
Acut

## Sepsis search

In [62]:
sepsis_icds = d_diagnoses_df[d_diagnoses_df["long_title"].str.contains("sepsis")]["icd_code"].compute().values

In [63]:
sepsis_hadm_ids = diagnoses_df[diagnoses_df["icd_code"].isin(sepsis_icds)]["hadm_id"].compute().values

In [64]:
lab_events_df[lab_events_df["hadm_id"].isin(sepsis_hadm_ids)].compute()

/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
12088,12104,10001401,27012892.0,14203043,50868,NaN,2133-07-10 05:30:00,2133-07-10 09:07:00,___,14.0,mEq/L,10.0,18.0,NaN,ROUTINE,New reference range as of ___.
12089,12105,10001401,27012892.0,14203043,50882,NaN,2133-07-10 05:30:00,2133-07-10 09:07:00,22,22.0,mEq/L,22.0,32.0,NaN,ROUTINE,NaN
12090,12106,10001401,27012892.0,14203043,50893,NaN,2133-07-10 05:30:00,2133-07-10 09:07:00,8.1,8.1,mg/dL,8.4,10.3,abnormal,ROUTINE,NaN
12091,12107,10001401,27012892.0,14203043,50902,NaN,2133-07-10 05:30:00,2133-07-10 09:07:00,106,106.0,mEq/L,96.0,108.0,NaN,ROUTINE,NaN
12092,12108,10001401,27012892.0,14203043,50912,NaN,2133-07-10 05:30:00,2133-07-10 09:07:00,1.6,1.6,mg/dL,0.4,1.1,abnormal,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523292,118326769,19997886,20793010.0,97694475,50983,NaN,2186-12-10 07:58:00,2186-12-10 11:06:00,150,150.0,mEq/L,135.0,147.0,abnormal,STAT,NaN
523293,118326770,19997886,20793010.0,97694475,51006,NaN,2186-12-10 07:58:00,2186-12-10 11:06:00,28,28.0,mg/dL,6.0,20.0,abnormal,STAT,NaN
523294,118326771,19997886,20793010.0,97694475,51678,NaN,2186-12-10 07:58:00,2186-12-10 11:06:00,4,4.0,NaN,NaN,NaN,NaN,STAT,NaN
523295,118326772,19997886,20793010.0,71557188,50813,NaN,2186-12-10 10:32:00,2186-12-10 10:35:00,___,2.1,mmol/L,0.5,2.0,abnormal,NaN,GREEN TOP TUBE.


In [65]:
d_lab_items[d_lab_items["itemid"] == 50868].compute()

,itemid,label,fluid,category
66,50868,Anion Gap,Blood,Chemistry


In [137]:
hemtology_features = [int(i) for i in ["51301", "51279", "51265", "51222", "51250"]]
hematology_lab_items = d_lab_items[d_lab_items["itemid"].isin(hemtology_features)]["itemid"].compute().values
d_lab_items[d_lab_items["itemid"].isin(hemtology_features)].compute()

,itemid,label,fluid,category
407,51222,Hemoglobin,Blood,Hematology
435,51250,MCV,Blood,Hematology
450,51265,Platelet Count,Blood,Hematology
464,51279,Red Blood Cells,Blood,Hematology
486,51301,White Blood Cells,Blood,Hematology


In [67]:
hematology_lab_events = lab_events_df[lab_events_df["itemid"].isin(hematology_lab_items)].compute()

/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [68]:
sepsis_hematology_lab_events = hematology_lab_events[hematology_lab_events["hadm_id"].isin(sepsis_hadm_ids)]

In [69]:
sepsis_hematology_lab_events

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
12103,12119,10001401,27012892.0,71368376,51222,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,9.8,9.80,g/dL,11.2,15.7,abnormal,ROUTINE,NaN
12106,12122,10001401,27012892.0,71368376,51250,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,99,99.00,fL,82.0,98.0,abnormal,ROUTINE,NaN
12107,12123,10001401,27012892.0,71368376,51265,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,224,224.00,K/uL,150.0,400.0,NaN,ROUTINE,NaN
12109,12125,10001401,27012892.0,71368376,51279,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,3.23,3.23,m/uL,3.9,5.2,abnormal,ROUTINE,NaN
12110,12126,10001401,27012892.0,71368376,51301,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,16.5,16.50,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523262,118326738,19997886,20793010.0,75222879,51222,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,8.4,8.40,g/dL,13.7,17.5,abnormal,ROUTINE,NaN
523265,118326741,19997886,20793010.0,75222879,51250,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,97,97.00,fL,82.0,98.0,NaN,ROUTINE,NaN
523266,118326742,19997886,20793010.0,75222879,51265,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,106,106.00,K/uL,150.0,400.0,abnormal,ROUTINE,NaN
523268,118326744,19997886,20793010.0,75222879,51279,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,2.80,2.80,m/uL,4.6,6.1,abnormal,ROUTINE,NaN


In [100]:
##join patients_df gender and anchor_age with lab events
joined_df = pd.merge(sepsis_hematology_lab_events, patients_df.compute(), on="subject_id")

In [101]:
joined_df

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,...,ref_range_lower,ref_range_upper,flag,priority,comments,gender,anchor_age,anchor_year,anchor_year_group,dod
0,12119,10001401,27012892.0,71368376,51222,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,9.8,9.80,...,11.2,15.7,abnormal,ROUTINE,NaN,F,89,2131,2014 - 2016,NaN
1,12122,10001401,27012892.0,71368376,51250,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,99,99.00,...,82.0,98.0,abnormal,ROUTINE,NaN,F,89,2131,2014 - 2016,NaN
2,12123,10001401,27012892.0,71368376,51265,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,224,224.00,...,150.0,400.0,NaN,ROUTINE,NaN,F,89,2131,2014 - 2016,NaN
3,12125,10001401,27012892.0,71368376,51279,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,3.23,3.23,...,3.9,5.2,abnormal,ROUTINE,NaN,F,89,2131,2014 - 2016,NaN
4,12126,10001401,27012892.0,71368376,51301,NaN,2133-07-10 05:30:00,2133-07-10 08:58:00,16.5,16.50,...,4.0,10.0,abnormal,ROUTINE,NaN,F,89,2131,2014 - 2016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835779,118326738,19997886,20793010.0,75222879,51222,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,8.4,8.40,...,13.7,17.5,abnormal,ROUTINE,NaN,M,67,2181,2011 - 2013,2186-12-11
835780,118326741,19997886,20793010.0,75222879,51250,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,97,97.00,...,82.0,98.0,NaN,ROUTINE,NaN,M,67,2181,2011 - 2013,2186-12-11
835781,118326742,19997886,20793010.0,75222879,51265,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,106,106.00,...,150.0,400.0,abnormal,ROUTINE,NaN,M,67,2181,2011 - 2013,2186-12-11
835782,118326744,19997886,20793010.0,75222879,51279,NaN,2186-12-10 03:51:00,2186-12-10 04:18:00,2.80,2.80,...,4.6,6.1,abnormal,ROUTINE,NaN,M,67,2181,2011 - 2013,2186-12-11


In [102]:
joined_df[joined_df.loc[:, "hadm_id"] == 29760041.0]

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,...,ref_range_lower,ref_range_upper,flag,priority,comments,gender,anchor_age,anchor_year,anchor_year_group,dod
43972,6703438,10590984,29760041.0,35996792,51222,NaN,2172-04-06 06:20:00,2172-04-06 07:34:00,10.2,10.20,...,13.7,17.5,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43973,6703441,10590984,29760041.0,35996792,51250,NaN,2172-04-06 06:20:00,2172-04-06 07:34:00,99,99.00,...,82.0,98.0,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43974,6703442,10590984,29760041.0,35996792,51265,NaN,2172-04-06 06:20:00,2172-04-06 07:34:00,138,138.00,...,150.0,400.0,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43975,6703444,10590984,29760041.0,35996792,51279,NaN,2172-04-06 06:20:00,2172-04-06 07:34:00,3.19,3.19,...,4.6,6.1,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43976,6703445,10590984,29760041.0,35996792,51301,NaN,2172-04-06 06:20:00,2172-04-06 07:34:00,20.3,20.30,...,4.0,10.0,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43977,6703471,10590984,29760041.0,53711200,51222,NaN,2172-04-07 06:18:00,2172-04-07 06:56:00,10.9,10.90,...,13.7,17.5,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43978,6703474,10590984,29760041.0,53711200,51250,NaN,2172-04-07 06:18:00,2172-04-07 06:56:00,99,99.00,...,82.0,98.0,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43979,6703475,10590984,29760041.0,53711200,51265,NaN,2172-04-07 06:18:00,2172-04-07 06:56:00,150,150.00,...,150.0,400.0,NaN,STAT,NaN,M,91,2172,2017 - 2019,NaN
43980,6703477,10590984,29760041.0,53711200,51279,NaN,2172-04-07 06:18:00,2172-04-07 06:56:00,3.40,3.40,...,4.6,6.1,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN
43981,6703478,10590984,29760041.0,53711200,51301,NaN,2172-04-07 06:18:00,2172-04-07 06:56:00,17.7,17.70,...,4.0,10.0,abnormal,STAT,NaN,M,91,2172,2017 - 2019,NaN


In [124]:
joined_df['counter'] = joined_df.groupby('specimen_id').cumcount()
pivot_df = joined_df.pivot_table(index='specimen_id',columns='itemid', values="valuenum", aggfunc='first')
pivot_df = pivot_df.reset_index()
pivot_df.columns.name = None
pivot_df

,specimen_id,51222,51250,51265,51279,51301
0,1868,7.9,85.0,343.0,2.95,6.3
1,2065,6.3,95.0,27.0,2.05,1.3
2,2173,10.3,108.0,44.0,2.98,11.0
3,3751,8.0,93.0,116.0,2.46,35.5
4,3846,7.8,97.0,138.0,2.46,15.5
...,...,...,...,...,...,...
171377,99997007,7.3,93.0,295.0,2.46,9.6
171378,99997579,11.2,88.0,128.0,3.54,24.4
171379,99998518,8.7,95.0,133.0,2.86,11.3
171380,99999395,6.9,95.0,26.0,2.38,4.3


In [139]:
merged_df = pd.merge(pivot_df, joined_df, on="specimen_id").loc[:, ["specimen_id","gender", "anchor_age", 51301, 51279, 51265, 51222, 51250]].drop_duplicates(subset="specimen_id")
merged_df.rename(columns={51222: "HGB", 51250: "MCV", 51265: "PLT", 51279:"RBC", 51301: "WBC"})

,specimen_id,gender,anchor_age,WBC,RBC,PLT,HGB,MCV
0,1868,M,56,6.3,2.95,343.0,7.9,85.0
5,2065,M,75,1.3,2.05,27.0,6.3,95.0
10,2173,M,59,11.0,2.98,44.0,10.3,108.0
15,3751,F,52,35.5,2.46,116.0,8.0,93.0
20,3846,F,78,15.5,2.46,138.0,7.8,97.0
...,...,...,...,...,...,...,...,...
834432,99997007,M,65,9.6,2.46,295.0,7.3,93.0
834437,99997579,M,68,24.4,3.54,128.0,11.2,88.0
834442,99998518,F,68,11.3,2.86,133.0,8.7,95.0
834447,99999395,M,43,4.3,2.38,26.0,6.9,95.0


In [ ]:
51222	Hemoglobin	Blood	Hematology
435	51250	MCV	Blood	Hematology
450	51265	Platelet Count	Blood	Hematology
464	51279	Red Blood Cells	Blood	Hematology
486	51301	White Blood Cells	Blood	Hematology

## Sepsis II

In [58]:
d_diagnoses_df[d_diagnoses_df["icd_code"].isin(["78552", "99591", "99592",
        "A021", "A207", "A227", "A241", "A267", "A327", "A392", "A393", "A394",
        "A40", "A400", "A401", "A403", "A408", "A409", "A41", "A410", "A4101",
        "A4102", "A411", "A412", "A413", "A414", "A4150", "A4151", "A4152",
        "A4153", "A4159", "A418", "A4181", "A4189", "A419", "A427", "B377",
        "R6520", "R6521"])].compute()

,icd_code,icd_version,long_title
9367,78552,9,Septic shock
12078,99591,9,Sepsis
12079,99592,9,Severe sepsis
12257,A021,10,Salmonella sepsis
12398,A207,10,Septicemic plague
12413,A227,10,Anthrax sepsis
12425,A241,10,Acute and fulminating melioidosis
12435,A267,10,Erysipelothrix sepsis
12470,A327,10,Listerial sepsis
12514,A392,10,Acute meningococcemia


In [66]:
d_diagnoses_df[d_diagnoses_df["icd_code"].isin(["99590", "R651", "R6510", "R6511"])].compute()

,icd_code,icd_version,long_title
12077,99590,9,"Systemic inflammatory response syndrome, unspe..."
42989,R651,10,Systemic inflammatory response syndrome (SIRS)...
42990,R6510,10,Systemic inflammatory response syndrome (SIRS)...
42991,R6511,10,Systemic inflammatory response syndrome (SIRS)...


In [60]:
hemato_events = lab_events_df[lab_events_df["itemid"].isin([  51301, # White Blood Cells, K/uL, Blood, Hematology, LOINC 804-5
            51279, # Red Blood Cells, m/uL, Blood, Hematology, LOINC 789-8
            51265, # Platelet Count, K/uL, Blood, Hematology, LOINC 777-3
            51222, # Hemoglobin, g/dL, Blood, Hematology, LOINC 718-7,
            51250  # MCV, fL, Blood, Hematology, LOINC 787-2
                             ])]

In [61]:
hemato_events = hemato_events.drop_duplicates(subset=["subject_id", "hadm_id", "charttime", "itemid"])

In [65]:
hemato_events.head()


KeyboardInterrupt



In [62]:
pivot_hemato_df = hemato_events.pivot_table(index=["subject_id", "hadm_id", "charttime"],columns='itemid', values="valuenum", aggfunc='first')

ValueError: 'index' must be the name of an existing column

In [ ]:
pivot_hemato_df = pivot_hemato_df.rename(columns={51222: "HGB", 51250: "MCV", 51265: "PLT", 51279:"RBC", 51301: "WBC"})

In [ ]:
pivot_hemato_df.loc[:, "HGB"] = pivot_hemato_df.loc[:, "HGB"]*0.621

In [171]:
pivot_hemato_df

itemid                                        HGB    MCV    PLT   RBC   WBC
subject_id hadm_id    charttime                                            
10000032   22595853.0 2180-05-07 05:05:00  7.8867   99.0   71.0  3.80   4.2
           22841357.0 2180-06-27 05:10:00  7.7004   99.0  137.0  3.60   6.6
           25742920.0 2180-08-06 06:36:00  7.5141  103.0  133.0  3.36   7.5
                      2180-08-07 06:15:00  7.2036  103.0  107.0  3.27   5.6
           29079034.0 2180-07-24 06:35:00  7.3899  102.0   94.0  3.40   4.1
...                                           ...    ...    ...   ...   ...
19999987   23865745.0 2145-11-07 06:00:00  7.1415  106.0  129.0  3.32   5.0
                      2145-11-03 01:35:00  8.1972  104.0  121.0  3.83  12.9
                      2145-11-04 10:40:00  7.8246  105.0  120.0  3.64  11.6
                      2145-11-05 06:10:00  7.6383  105.0  135.0  3.58  10.0
                      2145-11-06 10:07:00  7.8867  106.0  141.0  3.63   5.9

[1820760 rows x 5 columns]

In [173]:
transfer_df = transfer_df[transfer_df.loc[:, "eventtype"] != "discharge"]

In [175]:
transfer_df.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
5,10000032,22841357.0,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00
6,10000032,25742920.0,35509340,admit,Transplant,2180-08-06 01:44:00,2180-08-07 17:50:44


In [140]:
import time

for i in range(10000):
    time.sleep(60)
    print("Sleep")

Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep
Sleep


KeyboardInterrupt: 

In [79]:
import pandas as pd

# Example dataframe
data = {'A': ['ID1', 'ID2', 'ID1', 'ID3', 'ID2', 'ID4'],
        'B': [10, 20, 30, 40, 50, 60]}
df = pd.DataFrame(data)

# Create a new column as a counter for each unique value in column 'A'
df['counter'] = df.groupby('A').cumcount()

# Pivot the dataframe to convert values of column 'B' into separate columns
pivot_df = df.pivot_table(index='A', columns='counter', values='B', aggfunc='first')

# Reset the index and rename the columns
pivot_df = pivot_df.reset_index()
pivot_df.columns.name = None

# Print the new dataframe
print(pivot_df)


     A     0     1
0  ID1  10.0  30.0
1  ID2  20.0  50.0
2  ID3  40.0   NaN
3  ID4  60.0   NaN
